This notebook allows you to open an image and annotate polygonal regions that should be excluded from analysis—such as warped tissue areas, major imaging artifacts, or other unwanted regions.

In [3]:
import os
import sys
import napari
import dask.array as da
import numpy as np
from tifffile import imread
import pickle as pkl

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from sem_quant.load_config import load_config

## Specify config file

In [4]:
config_file_name = "../A02_config.json"

In [5]:
# read in config file

config = load_config(config_file_name)

im_path = config.paths.im_path
analysis_dir = config.paths.analysis_dir

output_prefix = config.paths.output_prefix
exclude_name = config.paths.exclude_file_suffix

axons_res = config.data_properties.axons_res

In [6]:
# create analysis directory if it doesn't exist
os.makedirs(analysis_dir, exist_ok=True)

## Open in the viewer and annotate

In [7]:
store = imread(im_path, aszarr=True)
im = da.from_zarr(store, axons_res)

In [8]:
viewer = napari.Viewer()
viewer.add_image(im)
viewer.add_shapes(name = exclude_name)

<Shapes layer 'discard_regions' at 0x1c1a7ed3710>

## Save

In [9]:
# collect data
discard_regions = viewer.layers[exclude_name].data

# save
with open(f'{analysis_dir}/{output_prefix}{exclude_name}.pkl', 'wb') as f:
    pkl.dump(discard_regions, f)